In [11]:
import random
from sklearn.metrics import precision_score, recall_score, f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [12]:
class Node:
    def __init__(self, language, words):
        self.language = language
        self.words = words
        self.translations = {}

    def add_translation(self, word, translated_word):
        self.translations[word] = translated_word

In [13]:
class LangChain:
    def __init__(self):
        self.nodes = {}

    def add_node(self, language, words):
        node = Node(language, words)
        self.nodes[language] = node

    def connect_nodes(self, lang1, lang2, word_pairs):
        for word, translated_word in word_pairs:
            self.nodes[lang1].add_translation(word, translated_word)
            self.nodes[lang2].add_translation(translated_word, word)

    def get_translation(self, language, word):
        return self.nodes[language].translations.get(word, None)

    def evaluate(self, test_data):
        predicted_translations = []
        true_translations = []

        for source_word, true_translation in test_data:
            predicted_translation = self.get_translation("English", source_word)
            predicted_translations.append(predicted_translation)
            true_translations.append(true_translation)

        precision = precision_score(true_translations, predicted_translations, average='micro')
        recall = recall_score(true_translations, predicted_translations, average='micro')
        f1 = f1_score(true_translations, predicted_translations, average='micro')

        # SmoothingFunction 적용
        smoothie = SmoothingFunction().method4
        bleu = sentence_bleu([true_translations], predicted_translations, smoothing_function=smoothie)

        return precision, recall, f1, bleu

In [14]:
# 예시 사용법
chain = LangChain()
chain.add_node("English", ["apple", "book", "cat"])
chain.add_node("Korean", ["사과", "책", "고양이"])

word_pairs = [("apple", "사과"), ("book", "책"), ("cat", "고양이")]
chain.connect_nodes("English", "Korean", word_pairs)

# 테스트 데이터셋 (예시로 동일한 데이터셋을 사용)
test_data = [("apple", "사과"), ("book", "책"), ("cat", "고양이")]

precision, recall, f1, bleu = chain.evaluate(test_data)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"BLEU Score: {bleu:.2f}")

Precision: 1.00
Recall: 1.00
F1-Score: 1.00
BLEU Score: 0.58
